In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the dataset
data = pd.read_csv('/content/Updated_Crop_Price_Prediction_Dataset.csv')

In [ ]:
# Selecting relevant columns for prediction
relevant_columns = [
    "Crop Name", "Rainfall (mm)", "Temperature (°C)", "Humidity (%)",
    "Soil pH", "Irrigation Availability (%)", "Fertilizer Type",
    "Growth per Day (cm/day)", "Fertilizer Cost (₹/kg)"
]
data_filtered = data[relevant_columns]

In [ ]:
# Encoding categorical variables
data_encoded = pd.get_dummies(data_filtered, columns=["Crop Name", "Fertilizer Type"], drop_first=True)

In [ ]:
# Splitting the data into features and target variable
X = data_encoded.drop(columns=["Growth per Day (cm/day)"])
y_growth = data_encoded["Growth per Day (cm/day)"]


In [ ]:
# Adding computed targets for water and fertilizer requirements
data_encoded['Water Requirement (L/ha)'] = data_encoded['Rainfall (mm)'] * 10  # Example calculation
data_encoded['Fertilizer Requirement (kg/ha)'] = data_encoded['Fertilizer Cost (₹/kg)'] * 2  # Example calculation
y_water = data_encoded["Water Requirement (L/ha)"]
y_fertilizer = data_encoded["Fertilizer Requirement (kg/ha)"]


In [ ]:
# Standardizing the features to improve model performance
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
# Splitting into training and testing sets
X_train_scaled, X_test_scaled, y_growth_train, y_growth_test = train_test_split(X_scaled, y_growth, test_size=0.2, random_state=42)
X_train_water, X_test_water, y_water_train, y_water_test = train_test_split(X_scaled, y_water, test_size=0.2, random_state=42)
X_train_fertilizer, X_test_fertilizer, y_fertilizer_train, y_fertilizer_test = train_test_split(X_scaled, y_fertilizer, test_size=0.2, random_state=42)

In [ ]:
# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10]
}

In [ ]:
# Model for growth prediction
model_growth = RandomForestRegressor(random_state=42)
grid_search_growth = GridSearchCV(model_growth, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search_growth.fit(X_train_scaled, y_growth_train)
best_model_growth = grid_search_growth.best_estimator_

# Model for water prediction
model_water = RandomForestRegressor(random_state=42)
grid_search_water = GridSearchCV(model_water, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search_water.fit(X_train_water, y_water_train)
best_model_water = grid_search_water.best_estimator_

# Model for fertilizer prediction
model_fertilizer = RandomForestRegressor(random_state=42)
grid_search_fertilizer = GridSearchCV(model_fertilizer, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search_fertilizer.fit(X_train_fertilizer, y_fertilizer_train)
best_model_fertilizer = grid_search_fertilizer.best_estimator_

In [ ]:
# Evaluate models
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    rmse = mse ** 0.5
    return mse, r2, rmse

# Evaluate each model
mse_growth, r2_growth, rmse_growth = evaluate_model(best_model_growth, X_test_scaled, y_growth_test)
mse_water, r2_water, rmse_water = evaluate_model(best_model_water, X_test_scaled, y_water_test)
mse_fertilizer, r2_fertilizer, rmse_fertilizer = evaluate_model(best_model_fertilizer, X_test_scaled, y_fertilizer_test)

# Print evaluation results
print("Growth Prediction - MSE:", mse_growth, "R2:", r2_growth, "RMSE:", rmse_growth)
print("Water Requirement Prediction - MSE:", mse_water, "R2:", r2_water, "RMSE:", rmse_water)
print("Fertilizer Requirement Prediction - MSE:", mse_fertilizer, "R2:", r2_fertilizer, "RMSE:", rmse_fertilizer)

Growth Prediction - MSE: 0.15715651585389737 R2: -0.16851291875508068 RMSE: 0.39642971111395947
Water Requirement Prediction - MSE: 3372.196476124991 R2: 0.9953370556451456 RMSE: 58.070616288489575
Fertilizer Requirement Prediction - MSE: 4.763449173000234 R2: 0.9987937133887964 RMSE: 2.1825327427097707


In [ ]:
# Function for making predictions
crop_types = data["Crop Name"].unique()
fertilizer_types = data["Fertilizer Type"].unique()

def predict_crop_growth_and_requirements(crop_name, rainfall, temperature, humidity, soil_pH, irrigation, fertilizer_type):
    input_data = pd.DataFrame({
        "Rainfall (mm)": [rainfall],
        "Temperature (°C)": [temperature],
        "Humidity (%)": [humidity],
        "Soil pH": [soil_pH],
        "Irrigation Availability (%)": [irrigation]
    })

    # One-hot encoding for the crop and fertilizer type
    for crop in crop_types:
        input_data[f"Crop Name_{crop}"] = 1 if crop == crop_name else 0

    for fert in fertilizer_types:
        input_data[f"Fertilizer Type_{fert}"] = 1 if fert == fertilizer_type else 0

    # Ensuring column alignment with training data
    missing_cols = set(X.columns) - set(input_data.columns)
    for col in missing_cols:
        input_data[col] = 0

    input_data = input_data[X.columns]

    # Scaling input features
    input_scaled = scaler.transform(input_data)

    # Predicting growth per day and converting to cm/month
    growth_per_day = best_model_growth.predict(input_scaled)[0]
    growth_per_month = growth_per_day * 30

    # Predicting water and fertilizer requirements per hectare
    water_requirement = best_model_water.predict(input_scaled)[0]
    fertilizer_requirement = best_model_fertilizer.predict(input_scaled)[0]

    return {
        "Growth per Month (cm)": round(growth_per_month, 2),
        "Water Requirement (L/ha)": round(water_requirement, 2),
        "Fertilizer Requirement (kg/ha)": round(fertilizer_requirement, 2)
    }

In [ ]:
# Dynamic user input
crop_name = input("Enter the crop name (e.g., Wheat, Rice): ")
rainfall = float(input("Enter rainfall in mm: "))
temperature = float(input("Enter temperature in °C: "))
humidity = float(input("Enter humidity in %: "))
soil_pH = float(input("Enter soil pH level: "))
irrigation = float(input("Enter irrigation availability in %: "))
fertilizer_type = input("Enter fertilizer type (e.g., Urea, DAP): ")

# Predict based on user input
predicted_values = predict_crop_growth_and_requirements(
    crop_name=crop_name,
    rainfall=rainfall,
    temperature=temperature,
    humidity=humidity,
    soil_pH=soil_pH,
    irrigation=irrigation,
    fertilizer_type=fertilizer_type
)

print("Predicted Values:", predicted_values)

Enter the crop name (e.g., Wheat, Rice): maize
Enter rainfall in mm: 124
Enter temperature in °C: 42
Enter humidity in %: 45
Enter soil pH level: 7.5
Enter irrigation availability in %: 65
Enter fertilizer type (e.g., Urea, DAP): urea
Predicted Values: {'Growth per Month (cm)': np.float64(33.27), 'Water Requirement (L/ha)': np.float64(1205.3), 'Fertilizer Requirement (kg/ha)': np.float64(25.79)}
